In [ ]:
import opengrid as og
from opengrid.library import plotting as og_plot
from opengrid.library import utils as og_util
import pandas as pd
from joule import meta, filter_meta

import yaml
import datetime
from workalendar.europe import Belgium

In [ ]:
def get_openinghours(ea_number, ts):
    
    df = pd.Series(data=False, index=ts.index)
    
    # Read YAML file
    with open("data/" + ea_number + ".yml", 'r') as stream:
        data_loaded = yaml.load(stream)
        for day in data_loaded:
            for key in day:
                hour_list = day[key]
                for hours in hour_list:
                    df = df | week_schedule(index=ts.index, on_days=[key], on_time=hours[0], off_time=hours[1], closed=[datetime.date(2018, 4, 25)])
    
    return df

In [ ]:
def week_schedule(index, on_days, on_time=None, off_time=None, exclude_holidays=True, closed=None):
    cal = Belgium()
    
    if on_time is None:
        on_time = '9:00'
    if off_time is None:
        off_time = '17:00'
    
    if not isinstance(on_time, datetime.time):
        on_time = pd.to_datetime(on_time, format='%H:%M').time()
    if not isinstance(off_time, datetime.time):
        off_time = pd.to_datetime(off_time, format='%H:%M').time()
    
    times = (index.time >= on_time) & (index.time < off_time) & (index.weekday_name.isin(on_days))
    
    result = pd.Series(times, index=index)
    
    holidays = []
    years = list(range(index[0].year, index[-1].year + 1))
    for y in years:
        for d in cal.get_calendar_holidays(y):
             holidays.append(d[0])
    
    if exclude_holidays:
        result[result.index.map(lambda x : x.date() in holidays)] = False
        
    if closed is not None:
        if isinstance(closed[0], datetime.date):
            result[result.index.map(lambda x : x.date() in closed)] = False
        
    return result

In [ ]:
df = get_openinghours('EA-14123117', pd.read_pickle('data/Electricity_EA-14123117.pkl').sum(axis=1))

In [ ]:
# carpet
plt = og.plot_style()
og_plot.carpet(df*1)
plt.show()